# Asistente Turístico de Tenerife
**Práctica Final - Large Language Models**

In [1]:
# Section 1: Setup and PDF Loading
from src.conf import OPENAI_API_KEY, MODEL_CONFIG, PDF_PATH
from src.logger import logger
from src.api_client import OpenAIClient
from src.data_loader import DataLoader

print(f"API Key loaded: {OPENAI_API_KEY[:8]}...")
print(f"Model config: {MODEL_CONFIG}")

API Key loaded: sk-proj-...
Model config: {'model': 'gpt-4o-mini', 'temperature': 0.3, 'max_tokens': 1024, 'top_p': 0.9}


In [2]:
# Initialize OpenAI client
client = OpenAIClient()

if client.test_connection():
    print("✅ OpenAI connection OK")
else:
    print("❌ OpenAI connection failed")

2026-01-14 17:29:01,189 - INFO - OpenAI client initialized with model: gpt-4o-mini
2026-01-14 17:29:02,091 - INFO - OpenAI connection test successful


✅ OpenAI connection OK


In [3]:
# Load PDF
loader = DataLoader(PDF_PATH)
pages = loader.load()
stats = loader.get_stats()
print(f"✅ PDF loaded: {stats}")

2026-01-14 17:29:03,679 - INFO - DataLoader initialized with: data/TENERIFE.pdf
2026-01-14 17:29:04,000 - INFO - Loaded 25 pages from data/TENERIFE.pdf


✅ PDF loaded: {'num_pages': 25, 'total_chars': 16094, 'total_words': 2817, 'avg_chars_per_page': 643}


In [4]:
# Section 2: Vector Store
from src.vector_store import VectorStore

vector_store = VectorStore()
num_chunks = vector_store.build_from_documents(pages)
print(f"✅ Vector store built: {num_chunks} chunks")
print(f"Chunk stats: {vector_store.get_chunk_stats()}")

2026-01-14 17:29:08,776 - INFO - VectorStore initialized (chunk_size=1000, overlap=200)
2026-01-14 17:29:08,777 - INFO - Created 30 chunks from 25 pages
2026-01-14 17:29:09,800 - INFO - Vector store built and persisted to ./chroma_db


✅ Vector store built: 30 chunks
Chunk stats: {'num_chunks': 30, 'avg_chunk_size': 570, 'min_chunk_size': 86, 'max_chunk_size': 987}


In [6]:
# Section 3: RAG Chain - Retrieval + Generation
from src.rag_chain import RAGChain

rag = RAGChain(client, vector_store)
print("✅ RAG Chain initialized")

2026-01-14 17:30:17,484 - INFO - RAGChain initialized with prompt from src/system_prompt.txt


✅ RAG Chain initialized


In [7]:
# Test RAG query
result = rag.query("¿Cómo puedo subir al Teide?")

print("PREGUNTA:", result["question"])
print("\nRESPUESTA:")
print(result["answer"])
print("\nFUENTES:")
for s in result["sources"]:
    print(f"  - Página {s['page']}: {s['preview'][:50]}...")

2026-01-14 17:32:15,263 - INFO - Search query: '¿Cómo puedo subir al Teide?...' returned 3 results
2026-01-14 17:32:15,265 - INFO - Retrieved 3 chunks for: ¿Cómo puedo subir al Teide?...
2026-01-14 17:32:23,811 - INFO - Completion received - tokens used: 1238
2026-01-14 17:32:23,812 - INFO - Generated answer of 1225 chars


PREGUNTA: ¿Cómo puedo subir al Teide?

RESPUESTA:
Para subir al Teide, tienes varias opciones:

1. **Carretera TF24**: Una recomendación es tomar la carretera TF24 (carretera de La Esperanza) desde la rotonda de Padre Anchieta en La Laguna. Durante el trayecto, puedes hacer una parada en el **Mirador de La Tarta**, donde podrás disfrutar de unas vistas espectaculares, especialmente si hay nubes en el norte, ya que podrás ver un impresionante mar de nubes.

2. **Parador de las Cañadas del Teide**: Una vez que llegues al Parque Nacional, lo ideal es aparcar en el Parador de las Cañadas del Teide. Desde aquí, puedes caminar hacia el **mirador de La Ruleta**, que ofrece una de las vistas más icónicas de la isla.

3. **Teleférico del Teide**: Si deseas subir hasta el pico del Teide, puedes utilizar los teleféricos disponibles desde el área del Parador. 

4. **Centro de Visitantes de El Portillo**: Si buscas más información sobre el Teide, puedes visitar el Centro de Visitantes de El Portill

In [8]:
# Test another query
result = rag.query("¿Dónde puedo comer bien en el norte de Tenerife? Si hay arepas mejor")

print("PREGUNTA:", result["question"])
print("\nRESPUESTA:")
print(result["answer"])

2026-01-14 17:33:54,316 - INFO - Search query: '¿Dónde puedo comer bien en el norte de Tenerife? S...' returned 3 results
2026-01-14 17:33:54,317 - INFO - Retrieved 3 chunks for: ¿Dónde puedo comer bien en el norte de Tenerife? S...
2026-01-14 17:34:08,363 - INFO - Completion received - tokens used: 1332
2026-01-14 17:34:08,364 - INFO - Generated answer of 1880 chars


PREGUNTA: ¿Dónde puedo comer bien en el norte de Tenerife? Si hay arepas mejor

RESPUESTA:
¡Claro! En el norte de Tenerife tienes varias opciones deliciosas para comer, especialmente si te gustan las arepas. Aquí te dejo algunas recomendaciones:

1. **Arepera La Carajita**  
   - **Ubicación**: Entre La Orotava y el Puerto de la Cruz  
   - **Descripción**: Este lugar es famoso por tener las mejores arepas que podrás comer. Te recomiendo probar la "La Fogalera" y "Reina Pepiada". Además, no olvides pedir una degustación de cachapitas como entrante. ¡Importante ir con hambre!

2. **Atuvera**  
   - **Ubicación**: También entre La Orotava y el Puerto de la Cruz  
   - **Descripción**: Este restaurante es muy recomendable, pero es mejor hacer una reserva antes de ir.

3. **Pizzería Gioconda**  
   - **Ubicación**: Zona de La Orotava  
   - **Descripción**: Ideal si te apetece una buena pizza.

4. **El Palestra**  
   - **Ubicación**: Zona de La Orotava  
   - **Descripción**: Un clásico e

In [9]:
# Test with something NOT in the guide
result = rag.query("¿Cuál es la capital de Francia?")

print("PREGUNTA:", result["question"])
print("\nRESPUESTA:")
print(result["answer"])

2026-01-14 17:34:42,876 - INFO - Search query: '¿Cuál es la capital de Francia?...' returned 3 results
2026-01-14 17:34:42,879 - INFO - Retrieved 3 chunks for: ¿Cuál es la capital de Francia?...
2026-01-14 17:34:44,407 - INFO - Completion received - tokens used: 592
2026-01-14 17:34:44,408 - INFO - Generated answer of 46 chars


PREGUNTA: ¿Cuál es la capital de Francia?

RESPUESTA:
No tengo esa información en mi guía turística.
